In [1]:
import os
import json
from datetime import (date, datetime, time)

In [2]:
# File Loading

stringyfiedArchitecturesData = open('./Data/architecture_models.json', encoding="utf8")
architecturesData = json.load(stringyfiedArchitecturesData)

stringyfiedCharactersData = open('./Data/characters-creatures_models.json', encoding="utf8")
charactersData = json.load(stringyfiedCharactersData)

stringyfiedHeritageData = open('./Data/cultural-heritage-history_models.json',encoding="utf8")
heritageData = json.load(stringyfiedHeritageData)

#Example of a 3D object data descriptor
print(heritageData[0]['publishedAt'])

2016-08-23T16:51:09.378793


In [3]:
import numpy as np
import scipy as sc
import pandas as pd
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)

In [4]:
usedCategories = {
    "undefined": 0,
    "characters-creatures": 1,
    "architecture": 2,
    "cultural-heritage-history": 3
}

def loadToNDimArray(modelsData, categoryCode):
  #uid = []
  likeCount = []
  commentCount = []
  viewCount = []
  faceCount = []
  vertexCount = []
  category = []
  lifetime = []

  for model in modelsData:
    #uid.append(model["uid"])
    likeCount.append(int(model["likeCount"]))
    commentCount.append(int(model["commentCount"]))
    viewCount.append(int(model["viewCount"]))
    faceCount.append(int(model["faceCount"]))
    vertexCount.append(int(model["vertexCount"]))

    try:
      deltaDays = datetime(2020, 2, 25) - datetime.strptime(
      model["publishedAt"], '%Y-%m-%dT%H:%M:%S.%f')
      lifetime.append(deltaDays.days)
    except:
      deltaDays = datetime(2020, 2, 25) - datetime.strptime(
      model["publishedAt"], '%Y-%m-%dT%H:%M:%S')
      lifetime.append(deltaDays.days)

    category.append(int(categoryCode))

  #uid = np.array(uid) 
  likeCount = np.array(likeCount)
  commentCount = np.array(commentCount)
  viewCount = np.array(viewCount)
  faceCount = np.array(faceCount)
  vertexCount = np.array(vertexCount)
  category = np.array(category)

  return np.stack(( likeCount, commentCount, viewCount, faceCount, vertexCount,  lifetime, category), axis = 0).transpose((1,0))


In [5]:
# Processing information in numpy matrix
charactersDataTable = loadToNDimArray(charactersData, 1)
architecturesDataTable = loadToNDimArray(architecturesData, 2)
heritageDataTable = loadToNDimArray(heritageData, 3)

In [6]:
dataTable = np.append(architecturesDataTable, charactersDataTable, axis=0)
dataTable = np.append(dataTable,  heritageDataTable, axis=0)

In [7]:
collumns = ["Likes", "Comments", "Views", "FaceCount", "VertexCount", "Days", "Category"] 
globalDf =  pd.DataFrame(
    data=dataTable,
    columns= collumns
)

charactersDf =  pd.DataFrame(
    data=charactersDataTable,
    columns= collumns
)

architecturesDf =  pd.DataFrame(
    data=architecturesDataTable,
    columns= collumns
)

heritageDf =  pd.DataFrame(
    data=heritageDataTable,
    columns= collumns
)


In [8]:
def appendCalculatedColumns(df):
    df["LikeViewRateo"] = df["Likes"]/df["Views"]
    df["LikesEachDay"] = df["Likes"]/df["Days"]
    df["ViewsEachDay"] = df["Views"]/df["Days"]
    return df

In [9]:
heritageDf = appendCalculatedColumns(heritageDf)
globalDf = appendCalculatedColumns(globalDf)
charactersDf = appendCalculatedColumns(charactersDf)
architecturesDf = appendCalculatedColumns(architecturesDf)

In [10]:
'''
    Pandas dataframes:
        -heritageDf: contains the data relatives to the cultural and heritage category
        -charactersDf: contains the data relatives to the characters and creatures category
        -architecturesDf: contains the data relatives to the architecture category category

    Each dataframe has the following columns:
        -Likes : number of likes
        -Comments : number of comments
        -Views : number of views
        -FaceCount : number of faces
        -VertexCount : number of vertex
        -Days : days of uptime (relative to the download date 25-Feb-2020) 
        -Category : category identificator number (
            0:undefined, 
            1:characters-creatures, 
            2:architecture, 
            3:cultural-heritage-history
        )
        -LikesViewsRateo : Likes per views rateo ( calculated as Likes\Views )
        -LikesEachDay : likes each day rateo ( calculated as Likes\Days )
        -ViewsEachDay : views each day rateo ( calculated as Views\Days )
'''

'\n    Pandas dataframes:\n        -heritageDf: contains the data relatives to the cultural and heritage category\n        -charactersDf: contains the data relatives to the characters and creatures category\n        -architecturesDf: contains the data relatives to the architecture category category\n\n    Each dataframe has the following columns:\n        -Likes : number of likes\n        -Comments : number of comments\n        -Views : number of views\n        -FaceCount : number of faces\n        -VertexCount : number of vertex\n        -Days : days of uptime (relative to the download date 25-Feb-2020) \n        -Category : category identificator number (\n            0:undefined, \n            1:characters-creatures, \n            2:architecture, \n            3:cultural-heritage-history\n        )\n        -LikesViewsRateo : Likes per views rateo ( calculated as Likes\\Views )\n        -LikesEachDay : likes each day rateo ( calculated as Likes\\Days )\n        -ViewsEachDay : views